# IUM projekt - etap 2
### Jakub Grzechociński  
### Piotr Sawicki


## Treść zadania
> Są osoby, które wchodzą na naszą stronę i nie mogą się zdecydować, którym produktom przyjrzeć się nieco lepiej. Może dało by się im coś polecić?

## Streszczenie etapu pierwszego
Do stworzenia jest system oferujący usługę rekomendacji produktów dla klientów sklepu internetowego. System opiera swoje działanie na danych przejrzeń i zakupów produktów przez użytkowników sklepu oraz na danych samych produktów.

System oferuje rekomendacje na podstawie dwóch oddzielnych modeli:
- `popularnościowy` bazujący na względnej popularności produktów i kategorii oraz preferencji użytkownika
- `collaborative filtering` odtwarzający podobieństwa między klientami

W ramach implementacji systemu należy utworzyć interfejs dla modeli rekomendacji w postaci mikroserwisu. W wyniku jego działania aplikacje klienckie mogą uzyskiwać rekomendacje produktów na podstawie id użytkownika. System pozwala na działanie z wykorzystaniem jednego z dwóch modeli lub w trybie testu A/B dla porównania modeli.

W etapie pierwszym wykonana została analiza danych, w ramach której zidentyfikowane zostały problemy oraz oszacowano, że system jest w stanie skutecznie rekomendować na podstawie danych. Podczas wykonywania etapu drugiego okazało się, że w danych jest jednak błąd dotyczący dystrybucji kategorii w danych zakupów. Ta dokumentacja odnosi się do nowego, poprawionego zestawu danych.

## Zaimplementowane modele 

### Model popularnościowy 

Model opiera swoje działanie na względnej popularności przedmiotów oraz ich kategorii. Zliczana jest ilość wyświetleń oraz zakupów każdego z produktów, przy czym zakup ma mnożnik X9 (jest to jeden z hiperparametrów modelu). Produkty są sortowane zgodnie z tak oszacowaną popularnością wewnątrz swoich kategorii. 

Następnie na podstawie przeglądania i zakupów każdego klienta wyznaczona zostaje jego najczęściej wybierana kategoria. Pierwszymi rekomendacjami modelu będą najpopularniejsze produkty, których dany klient jeszcze nie widział z najczęściej wybieranej kategorii. Jeżeli do tej pory liczba rekomendacji nie spełnia wymagań, wyznaczona zostaje kolejna kategoria. Wybierana jest kategoria najbardziej zbliżona do poprzedniej, przy czym bliskość określona jest jako liczba wspólnych nadkategorii, więc np. kategoria „Gry; Gry Xbox” będzie bliższa kategorii „Gry; Gry PC” niż „RTV; Telewizory”. Jeżeli w tym kroku algorytmu do wyboru jest kilka, tak samo bliskich, kategorii, wybierana jest ta, z której klient najczęściej przeglądał i kupował produkty. Z nowo wybranej kategorii znów wybierane są produkty, których klient jeszcze nie przeglądał. Proces ten powtarza się do wyczerpania kategorii. Jeżeli nawet wtedy nie udało się wybrać wymaganej liczby rekomendacji, do rekomendacji zostaną dodane produkty, które klient już przeglądał, według tego samego algorytmu.  

Model ten nie zawiera elementów uczenia maszynowego. Jest to “deterministyczny” model bazujący na prostej statystyce.  

Jego wadami są: 
- mała umiejętność dopasowania rekomendacji pod konkretnego klienta 
- nieuwzględnianie nieodzwierciedlonych bezpośrednio w danych atrybutów produktów 
- praktyczny brak możliwości rekomendacji produktu spoza kategorii bezpośredniego zainteresowania użytkownika.  

Zaletami: 
- Prosta struktura modelu pozwala łatwo uzasadnić jego bazową skuteczność 
- Względna odporność na znaczące zmiany struktury danych i anomalii w danych 
- Praktyczny brak wymagań dotyczących nadzoru ze względu na małą ilość hiperparametrów i ich niezależność od ilości danych w bazie 

### Model collaborative filtering 

Model dopasowuje klientów do siebie na podstawie ich podobieństwa. Podobieństwo obliczane jest na podstawie przejrzeń i zakupów poszczególnych produktów między użytkownikami sklepu.  

Dla każdej pary klient-produkt wyznaczana jest „ocena”, która wynosi 2, jeżeli klient kupił dany produkt lub 1, jeżeli tylko go przeglądał. Następnie „oceny” dla par, które nie zostały wyznaczone, obliczane są za pomocą algorytmu SVD. Rekomendacje dla danego klienta wyznaczane są poprzez posortowanie obliczonych przez algorytm SVD ocen związanych z danym klientem malejąco i zwrócenie odpowiadającym im produktów.  W przypadku, gdy liczba rekomendacji nie spełnia wymagań, wybierane są produkty, które klient już widział. 

Porównanie do modelu popularnościowego: 

Zalety: 
- Duża umiejętność dopasowania do indywidualnych preferencji użytkownika na podstawie jego podobieństwa do innych użytkowników. 
- Uwzględnianie ukrytych atrybutów produktów poprzez ich niejawne grupowanie szacowane na podstawie preferencji użytkowników. 
- Zdolność do rekomendacji produktów spoza bezpośredniego zainteresowania klienta. 

Wady: 
- Skomplikowana struktura modelu z elementem typu “black-box” - trudno wyjaśnić przyczynę skuteczności modelu. Dowód zasadności działania modelu zakłada istnienie “dobrych” wag modelu niemożliwych do uzyskania analitycznie. 
- Restrukturyzacja danych lub występowanie w nich anomalii mogą spowodować widoczne pogorszenie skuteczności modelu 
- Wraz z napływem danych model wymagać może dopasowania hiperparametrów w celu utrzymania skuteczności 

## Porównanie skuteczności modeli 

Obiektywne wyznaczenie skuteczności modelu rekomendacji na danych offline jest trudne, ponieważ dane te zostały zebrane, gdy system działał bez rekomendacji. Możemy jednak sprawdzić, gdy model zostanie poinformowany tylko o części akcji klienta, czy wśród zwróconych rekomendacji znajdują się produkty, które klient przeglądał lub, co lepiej, kupił, ale model nie został o tym poinformowany. Test taki bada tak naprawdę tylko zdolności predykcyjne modelu, lecz zdolność ta ogólnie rzecz biorąc jest porządana także w modelach rekomendacji. 

Wyznaczenie skuteczności modeli w taki sposób możliwe jest używając skryptu `model_test.py`. Skrypt ten dla każdego klienta ogranicza wiedzę modelu do 10 pierwszych produktów, które klient przeglądał, zachowując całą wiedzę o pozostałych klientach, a następnie generuje rekomendacje na podstawie tak zmodyfikowanej bazy danych. Następnie dla każdego klienta wyznaczane są wartości precision@k i recall@k, gdzie k to zadana liczba rekomendacji do wyznaczenia przez model. Precision@k to stosunek liczby rekomendowanych przedmiotów, które zostały zakupione przez klienta (o czym model, ucząc się, nie wiedział), do liczby rekomendacji. Recall@k to stosunek liczby rekomendowanych przedmiotów, które zostały zakupione przez klienta do liczby różnych przedmiotów, które kupił. Ostateczna skuteczność modelu określana jest na podstawie obu wartości precision@k i recall@k uśrednionych dla ilości przeprowadzonych prób. 

Skrypt `model_test.py` pozwala na określenie średnich wartości precision@k i recall@k. Wybór modelu do przetestowania jest możliwy poprzez podanie pierwszego argumenty wywołania skryptu: ‘cf’ dla model collaborative filtering, ‘mp’ dla modelu popularnościowego lub ‘rand’ dla losowych rekomendacji. Drugim argumentem wywołania skryptu może być liczba rekomendacji, którą powinien określać model. Aby ustawić liczbę rekomendacji na np. 10, należy uruchomić skrypt z argumentem `-k10`. Argument `-j` określa, dla ilu klientów na raz liczyć wartości. Domyślne argumenty to `model = cf, k = 10 i j = 8`. W przypadku wyboru modelu wynik liczony jest jednokrotnie, określając precision@k i recall@k dla każdego klienta, podczas gdy w przypadku losowych rekomendacji wynik jest uśredniany po 10 przebiegach, tak aby wyniki podczas różnych uruchomień były do siebie zbliżone. 

Uruchamiając skrypt dla obu modeli i losowego wyznaczania 10 rekomendacji wyniki są następujące: 

Model | Precision@10 | Recall@10 
--- | --- | --- 
Rekomendacje losowe  | 0.03 | 0.04 
Popularnościowy  | 0.22 | 0.28 
Collaborative Filtering  | 0.65 | 0.86 

Model collaborative filtering o wiele lepiej, w porównaniu do podstawowego modelu, jest w stanie przewidzieć przyszłe akcje klientów mając dane o innych klientach oraz szczątkowe dane o przeszłych akcjach danego klienta. W prawdziwym sklepie rekomendacje wyznaczane przez ten model powinny więc prowadzić do zwiększenia liczby odwiedzin stron przedmiotów i zakupów w sklepie internetowym. 

## Strojenie hiperparametrów 

Strojenie hiperparametrów algorytmu SVD, użytego w modelu Collaborative Filtering, możliwe jest za pomocą skryptu `tune_parameters.py`, który w sposób losowy dobiera wartości parametrów i sprawdza wartości precision@10 i recall@10 modelu z dobranymi parametrami, ostatecznie drukując wartości, dla których suma precision@10 i recall@10 była największa. Model podczas strojenia jest uczony za pomocą połowy danych ze zbioru trenującego. Ostateczne wartości hiperparametrów zostały ustalone po próbie 1000 różnych kombinacji. 

## Mikroserwis 

W ramach wykonania zadania utworzony został mikroserwis oferujący usługę rekomendacji dla użytkowników. Serwis składa się z pełnowymiarowego rozwiązania w postaci oprogramowania zarządzającego pobieraniem i czyszczeniem danych, uruchamianiem skryptów generujących rekomendacje, serwerem HTTP wraz z handlerem w postaci oferty rekomendacji, oraz bazą danych do obsługi i zapisywania żądań podczas testów A/B. 

Serwis napisany jest w języku Go. Modele rekomendacji i skrypty czyszczące dane w języku Python są przez niego okresowo wywoływane w celu generacji nowych zestawów rekomendacji dla każdego użytkownika.  Rekomendacje dla każdego użytkownika z osobna zapisywane są w plikach `.json`. Wraz z nadejściem żądania serwis przeszukuje katalog w poszukiwania odpowiedniej rekomendacji. Jeśli rekomendacji nie ma serwis nie przekazuje rekomendacji. 

Interfejs serwisu jest zrealizowany na podstawie protokołu HTTP. Interfejs wykonany jest zgodnie ze strukturą RESTful oraz JSON API. Oprogramowanie klienckie chcąc otrzymać rekomendacje wysyła prośbę HTTP GET wskazując odpowiedni adres, id użytkownika oraz, opcjonalnie, ile rekomendacji chce uzyskać. 

Struktura zapytania: 
``` 
HTTP GET <service_address>/recommendations?user_id=<user_id>&num=<num_recommendations> 
``` 
W razie znalezienia rekomendacji dla użytkownika serwis odpowiada w formie zbliżonej do: 
``` 
HTTP/1.1 200 OK 
Content-Type: application/json 
{"type":"user","id":___,"attributes":{"recommendations":[___, ___, ...]}} 
``` 
Serwis oferuje trzy tryby działania definiujące, którego modelu rekomendacje powinny być serwowane: 
- Rekomendacje modelu popularnościowego 
- Rekomendacje modelu CF 
- Tryb testu A/B oferujący mieszane rekomendacje modeli 

W czasie działania trybu A/B serwis komunikuje się z bazą danych (w SQLite) w celu zapisu przypisania użytkownika do konkretnego modelu oraz znajdywania odpowiedniego modelu dla użytkowników nie zapisanych w bazie. Utrzymywana jest informacja o ilości użytkowników przypisanych do danego modelu w celu sprawiedliwego porównywania modeli (serwis dopasowuje nadchodzące żądania nowych użytkowników tak, aby oba modele miały po tyle samo użytkowników). Informacja o przypisanym modelu jest zapisywana dla każdego z użytkowników z osobna, a przypisany model pozostaje ten sam, dopóki manualnie nie zresetowany zostanie test A/B.  Cały proces jest przezroczysty dla klientów sklepu (i oprogramowania klienckiego), aczkolwiek przy pierwszym połączeniu danego użytkownika czas odpowiedzi serwisu jest zauważalnie dłuższy (ale nieprzekraczający 0.2 sekundy), gdyż serwis musi ustalić i zapisać, do którego modelu przypisać użytkownika. 

Cały serwis opakowany jest w obraz kontenera Docker’a w celu jego prostego załadowania na serwer chmurowy. W plikach źródłowy zawarty jest `Dockerfile` pozwalający na zbudowanie obrazu, a sam obraz dostępny jest publicznie na serwisie [DockerHub](https://hub.docker.com/r/skwiwel/uni-recommendation-system). Obraz nie zawiera żadnych użytecznych danych. Dane wejściowe sklepu i baza danych A/B muszą być podmontowane do kontenera. 

Szczegóły uruchomieniowe serwisu znajdują się w pliku `Readme.md` w katalogu korzenia repozytorium. 

## Pokaz działania 

Serwis został umieszczony na maszynie w Oracle Cloud pod publicznym adresem ip `130.61.188.211`. Można więc przetestować jego działanie z dowolnego miejsca. Serwis odpowiada na odpowiednio sformułowane żądanie HTTP.   

Np. Aby uzyskać otrzymać odpowiedź zawierającą zestaw dziesięciu rekomendacji dla klienta o id `302` wystarczy wysłać zapytanie HTTP GET na adres: 
``` 
 130.61.188.211/recommendations?user_id=302&num=10 
``` 
W linii poleceń można szybko wykonać takie zapytanie przy pomocy narzędzia `curl`: 
``` 
curl -i "130.61.188.211/recommendations?user_id=302&num=10" 
``` 
Odpowiedź serwera: 
``` 
HTTP/1.1 200 OK
Content-Type: application/json
Date: Thu, 01 Apr 2021 16:55:16 GMT
Content-Length: 110

{"type":"user","id":302,"attributes":{"recommendations":[1055,1043,1010,1290,1039,1077,1051,1285,1124,1301]}}
``` 

Serwer chodzący pod publicznym IP pracuje w trybie testu A/B. Przyporządkowanie użytkowników do modelu zapisywane jest w bazie danych SQLite. Po stronie serwera możemy sprawdzić, który system wygenerował ten zestaw rekomendacji: 
``` 
sqlite> SELECT * FROM user_to_model WHERE user_id == 302;
user_id|model_id
302|2
``` 
Następnie szybki wgląd w przypisanie `model_id` do zrozumiałej nazwy: 
``` 
sqlite> SELECT * FROM models;
model_id|name|user_count
1|Popularity|7
2|Collaborative|8 
``` 

Dla zapytania dla użytkowinika o `user_id` 302 otrzymaliśmy więc rekomendacje od systemu `Collaborative`, czyli modelu CF. Na podstawie tak zbieranych danych oraz dalszych danych zbieranych przez sklep internetowy można oszacować skuteczność obu modeli. 

Warto by było teraz sprawdzić jakie produkty zostały zarekomendowane. Są to produkty o `product_id`: 
```
1055,1043,1010,1290,1039,1077,1051,1285,1124,1301
``` 

In [19]:
import datetime
import numpy as np
import pandas as pd

data_dir = 'data/'
data_raw_dir = 'data_raw/'
products_file = 'products.jsonl'
sessions_file = 'sessions.jsonl'
products_filepath = data_raw_dir + products_file
sessions_filepath = data_dir + sessions_file
products_data = pd.read_json(products_filepath, convert_dates=False, lines=True)
products_data = products_data.drop(columns=['price'])
sessions_data = pd.read_json(sessions_filepath, convert_dates=False, lines=True)

In [20]:
product_list = [1055,1043,1010,1290,1039,1077,1051,1285,1124,1301]
# get only the products from list
recommended_products = products_data[products_data.product_id.isin(product_list)]
# index them in the order from the list
recommended_products.reindex(recommended_products.product_id.map({x: i for i, x in enumerate(product_list)}).sort_values().index)

,product_id,product_name,category_path
54,1055,Call of Duty Modern Warfare 3 (PC),Gry i konsole;Gry komputerowe
42,1043,Fight Night Champion (PS3),Gry i konsole;Gry na konsole;Gry PlayStation3
9,1010,BioShock 2 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360
289,1290,Philips SDV5120,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...
38,1039,LCD Asus VS197D,Komputery;Monitory;Monitory LCD
76,1077,Kyocera FS-C2026MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...
50,1051,Spec Ops The Line (PC),Gry i konsole;Gry komputerowe
284,1285,Opticum AX-800,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...
123,1124,Devil May Cry 4 (PC),Gry i konsole;Gry komputerowe
300,1301,Vivanco TVA 400,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...


Produkty najlepsze zdaniem modelu są wyżej, gorsze niżej. Model CF zdaje się polecać dość różne produkty, między innymi gry komputerowe i na pozostałe platformy.

Poniżej sprawdzamy jakie produkty użytkownik oglądał do tej pory. Warto wspomnieć, że ten konkretnie użytkownik został utworzony przez nas sztucznie na potrzeby tego testu.

In [21]:
target_user_id = 302
# Odrzucenie danych nie związanych z użytkownikiem o id 302
sessions_data_for_user = sessions_data.loc[sessions_data['user_id']==target_user_id]
# Złączenie danych sesji i produktów
session_join_products = sessions_data_for_user.join(products_data.set_index('product_id'), on='product_id', how='left')
session_join_products = session_join_products[['session_id', 'user_id', 'event_type', 'category_path', 'product_name']]
session_join_products

,session_id,user_id,event_type,category_path,product_name
181252,110659,302,VIEW_PRODUCT,Gry i konsole;Gry komputerowe,Bioshock (PC)
181253,110659,302,VIEW_PRODUCT,Gry i konsole;Gry komputerowe,Civilization 4 Warlords (PC)
181254,110659,302,VIEW_PRODUCT,Gry i konsole;Gry komputerowe,Civilization 5 Bogowie i Królowie (PC)
181255,110659,302,VIEW_PRODUCT,Gry i konsole;Gry komputerowe,The Elder Scrolls 5 Skyrim (PC)
181256,110659,302,VIEW_PRODUCT,Gry i konsole;Gry komputerowe,Bioshock Infinite (PC)
181257,110659,302,VIEW_PRODUCT,Gry i konsole;Gry komputerowe,Just Cause (PC)


Użytkownik oglądał więc wyłącznie gry komputerowe.

Model CF poleca produkty zgodnie z podobieństwem aktywności użytkownika do aktywności innych klientów w bazie danych. Może wydawać się dziwne, że model polecił temu klientowi produkty o jedynie niebezpośrednim związku z produktami przez niego oglądanymmi, jednak to są właśnie produkty, które cieszyły się zainteresowaniem innych użytkowników. Rekomendacje opierają się na danych w bazie, a zakładając, że dane dobrze odzwierciedlają aktywność użytkowników, to rekomendacje są prawidłowe.

Dla porównania można sprawdzić jeszcze rekomendacje modelu popularnościowego dla klienta o id `302`:

In [24]:
product_list = [1055, 1051, 1112, 1141, 1169, 1101, 1132, 1083, 1144, 1102]
# get only the products from list
recommended_products = products_data[products_data.product_id.isin(product_list)]
# index them in the order from the list
recommended_products.reindex(recommended_products.product_id.map({x: i for i, x in enumerate(product_list)}).sort_values().index)

,product_id,product_name,category_path
54,1055,Call of Duty Modern Warfare 3 (PC),Gry i konsole;Gry komputerowe
50,1051,Spec Ops The Line (PC),Gry i konsole;Gry komputerowe
111,1112,Singularity (PC),Gry i konsole;Gry komputerowe
140,1141,Król Futbolu Piłkarski Quiz (PC),Gry i konsole;Gry komputerowe
168,1169,Bus Driver (PC),Gry i konsole;Gry komputerowe
100,1101,Dragonshard (PC),Gry i konsole;Gry komputerowe
131,1132,A New Beginning (PC),Gry i konsole;Gry komputerowe
82,1083,MAJESTY ANTOLOGIA (PC),Gry i konsole;Gry komputerowe
143,1144,Street Fighter 4 (PC),Gry i konsole;Gry komputerowe
101,1102,Dream Pinball 3D (PC),Gry i konsole;Gry komputerowe


Rekomendacje wyglądają bardziej rozsądnie, gdyż polecają tylko gry komputerowe. To, co jest jednak rozsądne dla czytającego ten raport nie musi być wcale tym, co jest rozsądne dla użytkowników, których aktywność zarejestrowana jest w bazie danych.

Pomijając kwestie kategorii, widać, że wśród rekomendacji gier komputerowych modelu CF widnieją dwie najbardziej popularne gry komputerowe jakie polecił model popularnościowy. Wskazuje to, że model CF działa poprawnie, gdyż produkty popularne dla większości użytkowników powinny często występować w rekomendacjach modeli bazujących na Collaborative Filtering.